# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load
file_to_load = "WeatherData.csv"

# Read Purchasing File and store into Pandas data frame
vacation_df = pd.read_csv(file_to_load)
vacation_df.dropna()
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,0,Lorengau,50,PG,1578373923,73,-2.02,147.27,84.87,8.63
1,1,Marawi,100,PH,1578373924,81,8.00,124.29,82.00,11.99
2,2,Sucua,48,EC,1578373924,97,-2.46,-78.17,64.62,2.68
3,3,Puerto Ayora,25,EC,1578373924,82,-0.74,-90.35,75.99,1.01
4,4,Mataura,84,NZ,1578373924,70,-46.19,168.86,57.00,11.99
...,...,...,...,...,...,...,...,...,...,...
522,522,Kathu,100,ZA,1578374016,71,-27.70,23.05,69.01,11.59
523,523,Hongjiang,0,CN,1578374017,62,27.21,109.83,58.96,4.97
524,524,Newport,86,GB,1578374017,87,51.59,-3.00,46.99,11.41
525,525,Mount Gambier,40,AU,1578373812,34,-37.83,140.78,78.80,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = vacation_df["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
criteria_df = vacation_df[vacation_df['Max_Temp'].between(70, 80)]
criteria_df = criteria_df[criteria_df['Wind_Speed']<10]
criteria_df = criteria_df = criteria_df[criteria_df['Cloudiness'] == 0]
criteria_df = criteria_df[criteria_df['Lat']<24]
criteria_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
8,8,Salalah,0,OM,1578373925,69,17.01,54.10,77.00,5.82
178,178,Pathalgaon,0,IN,1578373957,33,22.56,83.46,71.22,3.60
253,253,Pochutla,0,MX,1578373975,86,15.74,-96.47,73.00,6.82
276,276,Ibra,0,OM,1578373978,62,22.69,58.55,72.07,6.51
294,294,Gariaband,0,IN,1578373982,33,20.63,82.06,79.59,1.59
356,356,Loikaw,0,MM,1578373995,39,19.67,97.21,75.92,5.30
451,451,Praia,0,BR,1578374008,88,-20.25,-43.81,71.60,3.36
506,506,Boa Vista,0,BR,1578374015,65,2.82,-60.67,77.00,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [15]:
hotel_df = criteria_df
                                 
hotel_df['Hotel_Name'] = ""

hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel_Name
8,8,Salalah,0,OM,1578373925,69,17.01,54.10,77.00,5.82,
178,178,Pathalgaon,0,IN,1578373957,33,22.56,83.46,71.22,3.60,
253,253,Pochutla,0,MX,1578373975,86,15.74,-96.47,73.00,6.82,
276,276,Ibra,0,OM,1578373978,62,22.69,58.55,72.07,6.51,
294,294,Gariaband,0,IN,1578373982,33,20.63,82.06,79.59,1.59,
356,356,Loikaw,0,MM,1578373995,39,19.67,97.21,75.92,5.30,
451,451,Praia,0,BR,1578374008,88,-20.25,-43.81,71.60,3.36,
506,506,Boa Vista,0,BR,1578374015,65,2.82,-60.67,77.00,6.93,


In [16]:
# params dictionary to update each iteration
params = {
    "radius": 7000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng to identify Hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel_Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [17]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel_Name
8,8,Salalah,0,OM,1578373925,69,17.01,54.10,77.00,5.82,Al Baleed Resort Salalah by Anantara
178,178,Pathalgaon,0,IN,1578373957,33,22.56,83.46,71.22,3.60,Hotel Arnav Regency
253,253,Pochutla,0,MX,1578373975,86,15.74,-96.47,73.00,6.82,Bahia de la Luna
276,276,Ibra,0,OM,1578373978,62,22.69,58.55,72.07,6.51,Golden Rays hotel
294,294,Gariaband,0,IN,1578373982,33,20.63,82.06,79.59,1.59,Gariyaband Rest House
356,356,Loikaw,0,MM,1578373995,39,19.67,97.21,75.92,5.30,Keinnara Loikaw
451,451,Praia,0,BR,1578374008,88,-20.25,-43.81,71.60,3.36,Pousada Detalhes de Minas
506,506,Boa Vista,0,BR,1578374015,65,2.82,-60.67,77.00,6.93,Ibis Styles Boa Vista


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map